## Experiment 3

## Formation of Bus Admittance Matrix

In [1]:
import numpy as np

### Input

In [2]:
n = 4
b = 6
t = n - 1
l = b - t
twigs = np.array([['d', 'e', 'f']])
input_data = np.array([[('a', 4, 1, 2)], [('b', 3, 4, 2)], [('c', 1, 3, 5)], [('d', 1, 2, 5)], [('e', 2, 3, 5j)], [('f', 2, 4, -4j)]], dtype = [('ed', '<U4'), ('from', 'int'), ('to', 'int'), ('z', 'complex')])
links = []
# creating links for co-tree
for i in range(b):
    curr_branch = input_data['ed'][i][0]
    if curr_branch not in twigs:
        links.append(curr_branch)
links = np.array([links])

In [3]:
input_data

array([[('a', 4, 1,  2.+0.j)],
       [('b', 3, 4,  2.+0.j)],
       [('c', 1, 3,  5.+0.j)],
       [('d', 1, 2,  5.+0.j)],
       [('e', 2, 3,  0.+5.j)],
       [('f', 2, 4, -0.-4.j)]],
      dtype=[('ed', '<U4'), ('from', '<i4'), ('to', '<i4'), ('z', '<c16')])

In [4]:
# converting branch names to numbers

for i in range(b):
    input_data['ed'][i][0] = (ord(input_data['ed'][i][0]) - ord('a') + 1)

for i in range(t):
    twigs[0][i]= ord(twigs[0][i]) - ord('a')
twigs = twigs.astype('int')

for i in range(l):
    links[0][i] = ord(links[0][i]) - ord('a')
links = links.astype('int')

In [5]:
input_data

array([[('1', 4, 1,  2.+0.j)],
       [('2', 3, 4,  2.+0.j)],
       [('3', 1, 3,  5.+0.j)],
       [('4', 1, 2,  5.+0.j)],
       [('5', 2, 3,  0.+5.j)],
       [('6', 2, 4, -0.-4.j)]],
      dtype=[('ed', '<U4'), ('from', '<i4'), ('to', '<i4'), ('z', '<c16')])

In [6]:
links

array([[0, 1, 2]])

In [7]:
twigs

array([[3, 4, 5]])

In [39]:
np.set_printoptions(precision=3)  #setting printing precision

### Forming Reduced Incidence Matrix

In [40]:
A = np.zeros((n, b), dtype = 'int')

In [41]:
for i in range(b):
    curr_branch = int(input_data['ed'][i][0]) - 1
    from_node = input_data['from'][i][0] - 1
    to_node = input_data['to'][i][0] - 1
    
    A[from_node][curr_branch] = 1    #leaving a node => +1
    A[to_node][curr_branch] = -1     #Entering a node => -1

In [42]:
A

array([[-1,  0,  1,  1,  0,  0],
       [ 0,  0,  0, -1,  1,  1],
       [ 0,  1, -1,  0, -1,  0],
       [ 1, -1,  0,  0,  0, -1]])

In [43]:
ref_node = 4

In [44]:
A = np.delete(A, ref_node - 1, axis = 0)

In [45]:
Al = np.zeros((n -1, l), dtype = 'int')
Ab = np.zeros((n -1, t), dtype = 'int')

for i in range(l):
    Al[:, i] = A[:, links[0][i]]   # picking those branches which are part of links

for i in range(t):
    Ab[:, i] = A[:, twigs[0][i]]  # picking those branches which are twigs for the tree

In [46]:
Al

array([[-1,  0,  1],
       [ 0,  0,  0],
       [ 0,  1, -1]])

In [47]:
Ab

array([[ 1,  0,  0],
       [-1,  1,  1],
       [ 0, -1,  0]])

## Output

### Creating Cut-Set Matrix

In [48]:
Cb = np.dot(np.linalg.inv(Ab), Al).astype('int')

In [49]:
Ub = np.eye(t, dtype = 'int')

In [50]:
C = np.concatenate((Cb, Ub), axis = 1)

In [51]:
C

array([[-1,  0,  1,  1,  0,  0],
       [ 0, -1,  1,  0,  1,  0],
       [-1,  1,  0,  0,  0,  1]])

### Creating Loop Matrix

In [52]:
Ul = np.eye(l, dtype = 'int')

In [53]:
Bl = -1*(Cb.T)

In [54]:
Bl = Bl.astype('int')

In [55]:
Bl

array([[ 1,  0,  1],
       [ 0,  1, -1],
       [-1, -1,  0]])

In [56]:
B = np.concatenate((Ul, Bl) , axis = 1)

In [57]:
B

array([[ 1,  0,  0,  1,  0,  1],
       [ 0,  1,  0,  0,  1, -1],
       [ 0,  0,  1, -1, -1,  0]])

### Creating Z and Y Matrix

In [58]:
zd = [i[0] for i in input_data['z']]

In [59]:
zd

[(2+0j), (2+0j), (5+0j), (5+0j), 5j, (-0-4j)]

In [60]:
Z = np.diag(zd)
Z

array([[ 2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  5.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  5.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+5.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.-4.j]])

In [61]:
Y = np.linalg.inv(Z)
Y

array([[0.5-0.j  , 0. -0.j  , 0. -0.j  , 0. -0.j  , 0. -0.j  , 0. -0.j  ],
       [0. -0.j  , 0.5-0.j  , 0. -0.j  , 0. -0.j  , 0. -0.j  , 0. -0.j  ],
       [0. -0.j  , 0. -0.j  , 0.2-0.j  , 0. -0.j  , 0. -0.j  , 0. -0.j  ],
       [0. -0.j  , 0. -0.j  , 0. -0.j  , 0.2-0.j  , 0. -0.j  , 0. -0.j  ],
       [0. +0.j  , 0. +0.j  , 0. +0.j  , 0. +0.j  , 0. -0.2j , 0. +0.j  ],
       [0. +0.j  , 0. +0.j  , 0. +0.j  , 0. +0.j  , 0. +0.j  , 0. +0.25j]])

### Creating Ybus and Zbus Matrix

In [62]:
Ybus = np.dot(A, np.dot(Y, A.T))
Ybus

array([[ 0.9+0.j  , -0.2+0.j  , -0.2+0.j  ],
       [-0.2+0.j  ,  0.2+0.05j,  0. +0.2j ],
       [-0.2+0.j  ,  0. +0.2j ,  0.7-0.2j ]])

In [63]:
Zbus = np.linalg.inv(Ybus)
Zbus

array([[1.379-0.216j, 0.944-0.715j, 0.263-0.256j],
       [0.944-0.715j, 4.128-1.868j, 0.121-1.349j],
       [0.263-0.256j, 0.121-1.349j, 1.062+0.196j]])

### Creating Zloop and Yloop Matrix

In [64]:
Zloop = np.dot(B, np.dot(Z, B.T))
Zloop

array([[ 7.-4.j,  0.+4.j, -5.+0.j],
       [ 0.+4.j,  2.+1.j,  0.-5.j],
       [-5.+0.j,  0.-5.j, 10.+5.j]])

In [65]:
Yloop = np.linalg.inv(Zloop)
Yloop

array([[0.155+0.054j, 0.066-0.064j, 0.112+0.004j],
       [0.066-0.064j, 0.234-0.049j, 0.08 +0.045j],
       [0.112+0.004j, 0.08 +0.045j, 0.123-0.02j ]])

### Creating Ybr and Zbr Matrix

In [66]:
Ybr = np.dot(C, np.dot(Y, C.T))
Ybr

array([[ 0.9+0.j  ,  0.2+0.j  ,  0.5+0.j  ],
       [ 0.2+0.j  ,  0.7-0.2j , -0.5+0.j  ],
       [ 0.5+0.j  , -0.5+0.j  ,  1. +0.25j]])

In [67]:
Zbr = np.linalg.inv(Ybr)
Zbr

array([[ 3.619-0.654j, -3.325+0.061j, -3.184+1.153j],
       [-3.325+0.061j,  4.948+1.025j,  4.007-0.519j],
       [-3.184+1.153j,  4.007-0.519j,  4.128-1.868j]])